In [ ]:
#Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D

# Cell 2: Helper Functions
def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_point(point: np.ndarray, tris: np.ndarray, normals: np.ndarray) -> bool:
    """Check if a point is inside the mesh using ray casting."""
    ray = np.array([1.0, 0.0, 0.0])  # Cast ray along x-axis
    intersections = 0
    
    for tri, normal in zip(tris, normals):
        # Check if ray intersects triangle
        v0, v1, v2 = tri
        edge1 = v1 - v0
        edge2 = v2 - v0
        
        # Calculate determinant
        h = np.cross(ray, edge2)
        det = np.dot(edge1, h)
        
        if abs(det) < 1e-6:  # Ray is parallel to triangle
            continue
            
        inv_det = 1.0 / det
        s = point - v0
        u = inv_det * np.dot(s, h)
        
        if u < 0.0 or u > 1.0:
            continue
            
        q = np.cross(s, edge1)
        v = inv_det * np.dot(ray, q)
        
        if v < 0.0 or u + v > 1.0:
            continue
            
        t = inv_det * np.dot(edge2, q)
        
        if t > 1e-6:  # Ray intersects triangle
            # Check if intersection is in front of point
            if np.dot(normal, ray) < 0:
                intersections += 1
    
    return intersections % 2 == 1  # Odd number of intersections means point is inside

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using ray casting."""
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)  # 36mm - 1 for 0-based indexing
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)  # 30mm - 1
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)  # 30mm - 1
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Sample points at voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    
    # Process in chunks to manage memory
    chunk_size = 1000
    for i in range(0, len(z_coords), chunk_size):
        z_chunk = z_coords[i:i + chunk_size]
        for j in range(0, len(y_coords), chunk_size):
            y_chunk = y_coords[j:j + chunk_size]
            for k in range(0, len(x_coords), chunk_size):
                x_chunk = x_coords[k:k + chunk_size]
                
                # Create meshgrid for this chunk
                Z, Y, X = np.meshgrid(z_chunk, y_chunk, x_chunk, indexing='ij')
                points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
                
                # Check each point
                for point in points:
                    if _ray_cast_point(point, tris, normals):
                        z, y, x = point - np.array([z_min, y_min, x_min])
                        grid[int(z), int(y), int(x)] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True  # Keep only middle section
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x  # y' = -x
        new_x = y   # x' = y
        
        # Adjust coordinates to be non-negative
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        
        # Update rotated grid
        rotated_grid[z, new_y, new_x] = True
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [1]:
import os
from pathlib import Path
from stl_processor import process_directory

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

Found 0 STL files in 3D_Files/Grafted CTs
Found 14 STL files in 3D_Files/Randomly Generated ICs
Processing 1.3244 2441.stl...
Estimated triangles: 32,349
Loading STL file...
Processing 41292 voxels using 12 cores...


ValueError: operands could not be broadcast together with shapes (32349,3) (11285,3) 

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)

In [ ]:
# Cell 1: Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import struct
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing as mp
from functools import partial
import time

def estimate_triangles_from_size(file_path):
    """Estimate number of triangles in an STL file based on its size."""
    file_size = os.path.getsize(file_path)
    if file_size < 1000:  # ASCII STL
        return file_size // 100
    else:  # Binary STL
        return (file_size - 84) // 50

def load_stl(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Return (normals, triangles)."""
    path = Path(path)
    raw = path.read_bytes()

    # Check if ASCII
    if raw[:5].lower() == b"solid" and b"facet normal" in raw[:200]:
        normals = []
        tris = []
        for line in raw.decode("ascii", errors="ignore").splitlines():
            tokens = line.strip().split()
            if tokens[:2] == ["facet", "normal"]:
                normals.append(list(map(float, tokens[2:])))
            elif tokens[:1] == ["vertex"]:
                tris.append(list(map(float, tokens[1:])))
        tris = np.asarray(tris, np.float32).reshape(-1, 3, 3)
        normals = np.repeat(np.asarray(normals, np.float32)[:, None, :], 3, 1)[:, 0]
        return normals, tris

    # Binary STL
    header = raw[:80]
    n_tri = struct.unpack("<I", raw[80:84])[0]
    dt = np.dtype([
        ("normals", np.float32, (3,)),
        ("v1", np.float32, (3,)),
        ("v2", np.float32, (3,)),
        ("v3", np.float32, (3,)),
        ("attr", np.uint16),
    ])
    body = np.frombuffer(raw, count=n_tri, offset=84, dtype=dt)
    normals = body["normals"]
    tris = np.stack([body["v1"], body["v2"], body["v3"]], axis=1)
    return normals, tris

def _ray_cast_chunk(chunk_points, tris, normals):
    """Process a chunk of points using vectorized operations."""
    results = np.zeros(len(chunk_points), dtype=bool)
    ray = np.array([1.0, 0.0, 0.0])
    
    # Vectorized triangle intersection checks
    for i, point in enumerate(chunk_points):
        # Calculate vectors for all triangles at once
        v0 = tris[:, 0]
        edge1 = tris[:, 1] - v0
        edge2 = tris[:, 2] - v0
        
        # Calculate determinants for all triangles
        h = np.cross(ray, edge2)
        det = np.sum(edge1 * h, axis=1)
        
        # Skip parallel triangles
        valid_det = np.abs(det) > 1e-6
        if not np.any(valid_det):
            continue
            
        # Calculate intersection parameters
        s = point - v0
        inv_det = 1.0 / det[valid_det]
        u = inv_det * np.sum(s * h[valid_det], axis=1)
        
        # Check if intersection is within triangle
        valid_u = (u >= 0.0) & (u <= 1.0)
        if not np.any(valid_u):
            continue
            
        q = np.cross(s, edge1[valid_det][valid_u])
        v = inv_det[valid_u] * np.sum(ray * q, axis=1)
        
        # Check if intersection is valid
        valid_v = (v >= 0.0) & (u[valid_u] + v <= 1.0)
        if not np.any(valid_v):
            continue
            
        t = inv_det[valid_u][valid_v] * np.sum(edge2[valid_det][valid_u][valid_v] * q[valid_v], axis=1)
        
        # Count intersections
        intersections = np.sum((t > 1e-6) & (np.sum(normals[valid_det][valid_u][valid_v] * ray, axis=1) < 0))
        results[i] = intersections % 2 == 1
        
    return results

def stl_to_voxel_array(path: str | Path) -> tuple[np.ndarray, np.ndarray]:
    """Convert STL to voxel array using parallel ray casting."""
    start_time = time.time()
    print("Loading STL file...")
    normals, tris = load_stl(path)
    
    # Get bounding box
    all_points = tris.reshape(-1, 3)
    x_min, y_min, z_min = np.floor(all_points.min(axis=0))
    x_max, y_max, z_max = np.ceil(all_points.max(axis=0))
    
    # Ensure we capture the full 36mm x 30mm x 30mm structure
    x_min = min(x_min, 0)
    x_max = max(x_max, 35)
    y_min = min(y_min, 0)
    y_max = max(y_max, 29)
    z_min = min(z_min, 0)
    z_max = max(z_max, 29)
    
    # Create voxel grid
    grid = np.zeros((int(z_max - z_min + 1), 
                    int(y_max - y_min + 1), 
                    int(x_max - x_min + 1)), dtype=bool)
    
    # Generate all voxel centers
    z_coords = np.arange(z_min + 0.5, z_max + 0.5)
    y_coords = np.arange(y_min + 0.5, y_max + 0.5)
    x_coords = np.arange(x_min + 0.5, x_max + 0.5)
    Z, Y, X = np.meshgrid(z_coords, y_coords, x_coords, indexing='ij')
    points = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    # Split points into chunks for parallel processing
    n_cores = mp.cpu_count()
    chunk_size = len(points) // n_cores
    chunks = [points[i:i + chunk_size] for i in range(0, len(points), chunk_size)]
    
    print(f"Processing {len(points)} voxels using {n_cores} cores...")
    
    # Process chunks in parallel
    with mp.Pool(n_cores) as pool:
        results = pool.map(partial(_ray_cast_chunk, tris=tris, normals=normals), chunks)
    
    # Combine results
    results = np.concatenate(results)
    grid.ravel()[results] = True
    
    # Remove loading plates (3mm at top and bottom)
    mask = np.zeros_like(grid)
    mask[3:27] = True
    grid &= mask
    
    # Create rotated grid
    rotated_grid = np.zeros_like(grid)
    filled_idx = np.column_stack(np.nonzero(grid)).astype(np.int32)
    
    # Rotate 90 degrees counterclockwise about z-axis
    for z, y, x in filled_idx:
        new_y = -x
        new_x = y
        new_y -= min(-x for _, _, x in filled_idx)
        new_x -= min(y for _, y, _ in filled_idx)
        rotated_grid[z, new_y, new_x] = True
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    voxel_centers = np.column_stack(np.nonzero(rotated_grid)).astype(np.int32)
    return voxel_centers, rotated_grid

def process_stl_file(stl_path, output_dir):
    """Process a single STL file and save its voxelized representation."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Estimate triangles
    estimated_triangles = estimate_triangles_from_size(stl_path)
    print(f"Estimated triangles: {estimated_triangles:,}")
    
    # Convert STL to voxel array
    centers, matrix = stl_to_voxel_array(stl_path)
    
    # Get filename without extension
    filename = Path(stl_path).stem
    
    # Save the numpy array
    np.save(os.path.join(output_dir, f"{filename}.npy"), matrix)
    
    # Create interactive 3D plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the voxels
    ax.voxels(matrix, facecolors='white', edgecolors='black', linewidth=0.3)
    
    # Set up the plot
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.set_box_aspect([1, 1, 1])
    plt.title(f"Voxelized Structure: {filename}\nEstimated triangles: {estimated_triangles:,}")
    
    # Enable interactive features
    ax.mouse_init()
    
    # Show the plot
    plt.show()

def process_directory(input_dir, output_dir):
    """Process all STL files in a directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    stl_files = list(Path(input_dir).glob("**/*.stl"))
    print(f"Found {len(stl_files)} STL files in {input_dir}")
    
    for stl_file in stl_files:
        print(f"Processing {stl_file.name}...")
        process_stl_file(str(stl_file), output_dir)

# Cell 3: Main Execution
base_dir = "3D_Files"

# Process Grafted CTs
grafted_dir = os.path.join(base_dir, "Grafted CTs")
grafted_output = os.path.join(grafted_dir, "numpy_arrays")
process_directory(grafted_dir, grafted_output)

# Process Randomly Generated ICs
random_dir = os.path.join(base_dir, "Randomly Generated ICs")
random_output = os.path.join(random_dir, "numpy_arrays")
process_directory(random_dir, random_output)